In [1]:
import pandas as pd
from custom_functions_scRBA import *

In [2]:
#### LOAD INPUTS AND PARAMETERS
# Stoichiometry
df_stoich = pd.read_excel('./model/RBA_stoichiometry.xlsx')
df_stoich.index = df_stoich.id.to_list()

# Load protein sequence lengths
df_pro = pd.read_excel('./input/PROTEIN_stoich_curation_2021-09-28.xlsx')
df_pro.index = df_pro.id.to_list()

# Ribosome efficiency (amino acids translated per active ribosome per second)
# (script will convert the unit to per hour) (obsolete / implement directly in GAMS file now)
# kribo = 10.5

# Dummy protein's length (assigned to be the median protein length)
NAA_dummy = 401

In [3]:
### Enzyme synthesis requirement coupled to metabolic reaction rate
idx = df_stoich[df_stoich.coupling_type == 'rxn_enz'].index
eqn_list = []; kapp_list = []; eqn_list_equality = [];

for i in idx:
    lhs = "v('ENZLOAD-" + df_stoich.id[i][4:] + "') * " + "kapp('" + i + "')"
    rhs = "%mu% * v('" + i + "')"
    eqn_list.append(lhs + ' =g= ' + rhs + ';')
    eqn_list_equality.append(lhs + ' =e= ' + rhs + ';')
    
eqn_idx = ['EnzCap'+str(i) for i in range(0, len(eqn_list))]
eqn_list = ['EnzCap'+str(i)+'.. ' + eqn_list[i] for i in range(0, len(eqn_list))]

with open('./model/RBA_enzCapacityConstraints_declares.txt', 'w') as f:
    f.write('\n'.join(eqn_idx))
with open('./model/RBA_enzCapacityConstraints_eqns.txt', 'w') as f:
    f.write('\n'.join(eqn_list))
    
eqn_list_equality = ['EnzCap'+str(i)+'.. ' + eqn_list_equality[i] for i in range(0, len(eqn_list_equality))]
with open('./model/RBA_enzCapacityConstraints_eqns_equality_version.txt', 'w') as f:
    f.write('\n'.join(eqn_list_equality))

In [4]:
### Write protein length file (NAA)
pro_list = []
for i in df_pro.index:
    pro_list.append("'PROSYN-" + df_pro.id[i] + "' " + str(len(df_pro.sequence[i]) - 1))
pro_list.append("'PROSYN-PROTDUMMY' " + str(NAA_dummy))

pro_list = ['/'] + pro_list + ['/']
with open('./model/RBA_proteinLength.txt', 'w') as f:
    f.write('\n'.join(pro_list))

In [5]:
### Write prosyn reaction
idx = [i for i in df_stoich.index if i[:7] == 'PROSYN-']
prosyn = ["'" + i + "'" for i in idx]
prosyn = ['/'] + prosyn + ['/']
with open('./model/RBA_rxns_prosyn.txt', 'w') as f:
    f.write('\n'.join(prosyn))